# Imports

In [ ]:
import sys

sys.path.append('..')

In [ ]:
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from helpers.data_read_write import flattened_exercise_to_csv, csv_to_dataframe
from helpers.paths import DATA_DIR

# Collect Data

In [ ]:
data = csv_to_dataframe(flattened_exercise_to_csv())
data.head()

# Exploration

## 1. Describe data
- `id` and `relationship_id` were pulled in as ints, when really they should be string. `id` may actually be better index
- `relationship_id` has no corresponding table. Double checked and readme, and not included. Should drop
- `education_num` goes up to 16 ~ bachelor's degree
- No one is unemployeed. Eveyone has worked at least 1 hour.
- May be data inconsistencies in `hours_week`, goes up to 99. Should check distribution.
- Everyone is seriously underpaid based on the `over_50k` bool

In [ ]:
data.describe() # Numeric cols

## 2. Data info
- No null objects. Still need to pull out question mark data

In [ ]:
data.info()

## 3. data distributions
- There are 10 people who Never-worked, but have hours_week > 0?
- `education_num` likely does not include education beyond a bachelor's degree
- comparing `education_num` vs `education_level` would liekly give an indication of possible data errors.
  - Really, only one of these columns is needed. Prefer `education_num` as it provides a point of reference between levels
- Not sure what `Married-AF-spouse` is
- Data source is biased towards `White`, `Male`, and `United-States` at levels that are not expected from the country
- `Workplace`, `occupation`, and `country` have empty values occassionally

In [ ]:
for col in str_cols:
    print("\n*********", col)
    print(data[col].value_counts())

In [ ]:
str_cols = data.select_dtypes(object).columns

for col in str_cols:
    num_attributes = data[col].nunique()
    if num_attributes > 15:
        print(col, "too many attributes to effectively show bar plot")
        continue
    sns.catplot(x=col, kind="count", palette="ch:.25", data=data, height=4, aspect=num_attributes/3).set_titles(col)

## 4. education_num vs education_level
- `education_num` + 3 seems to be equivalent to the Amercian number of years to achieve the `education_level`

In [ ]:
data[['education_num', 'education_level']].head(30)

In [ ]:
# Make sure the same values always map
assert data[['education_num', 'education_level']].drop_duplicates().shape[0] == data['education_num'].nunique()

## 5. Specificy data and clear out unknowns ('?')
- With more time, would do more research into how to handle. 
- but I'm having a eck of a time stripping out question marks

## 6. capital_gain vs capital_loss
- A capital loss is a negative gain. When cleaning this, we can turn it into one column. `capital`

In [ ]:
test_data = data.copy()
test_data['capital_sum'] = data['capital_gain'] + data['capital_loss']
test_data['capital_max'] = data[['capital_gain', 'capital_loss']].max(axis=1)
assert test_data[['capital_sum', 'capital_max']].T.drop_duplicates().shape[0] == 1

# Data Cleaning

In [ ]:
clean_data = data.copy()
clean_data = clean_data.set_index('id')
clean_data['grades_completed'] = clean_data['education_num'] + 3
clean_data['capital'] = clean_data['capital_gain'] - clean_data['capital_loss']
clean_data = clean_data.drop(['relationship_id', 'capital_gain', 'capital_loss'], axis=1)
assert clean_data.shape[0] == data.shape[0]
assert clean_data.shape[1] + 2 == data.shape[1]

In [ ]:
# Set as NBCovert so as to not accidentally overwrite
# save clean data
clean_data_file = DATA_DIR /'clean_data.pkl'
with clean_data_file.open(mode='wb') as pkl_file:
    pickle.dump(clean_data, pkl_file)